In [ ]:
!pip3 install autokeras

In [ ]:
# 라이브러리 로드
import pandas as pd
import tensorflow as tf
import autokeras as ak

# 데이터셋 다운로드
dataset = tf.keras.utils.get_file(
    fname="AirQualityUCI.csv",
    origin="https://archive.ics.uci.edu/ml/machine-learning-databases/00360/"
    "AirQualityUCI.zip",
    extract=True
)

# 판다스 데이터프레임으로 로드
dataset = pd.read_csv(dataset, sep=";")
# 데이터 형태 및 컬럼 출력
print(dataset.shape)
print(dataset.columns)

In [ ]:
# 불필요한 컬럼 삭제
dataset = dataset[dataset.columns[:-2]]
# 결측값 NA로 치환
dataset = dataset.dropna()
# 특정 값 치환
dataset = dataset.replace(",", ".")

In [ ]:
# 검증 데이터셋 분리 (30%)
val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

# 데이터 형태 출력
print(data_train.shape)
print(validation_data.shape)

In [ ]:
# 데이터 타입 형변환
data_x=data_train[["CO(GT)","PT08.S1(CO)","NMHC(GT)","C6H6(GT)", "PT08.S2(NMHC)","NOx(GT)","PT08.S3(NOx)","NO2(GT)","PT08.S4(NO2)", "PT08.S5(O3)", "T","RH",]].astype("float64")
data_x_val=validation_data[["CO(GT)","PT08.S1(CO)","NMHC(GT)","C6H6(GT)", "PT08.S2(NMHC)","NOx(GT)","PT08.S3(NOx)","NO2(GT)","PT08.S4(NO2)", "PT08.S5(O3)", "T","RH",]].astype("float64")
data_x_test=dataset[["CO(GT)","PT08.S1(CO)","NMHC(GT)","C6H6(GT)", "PT08.S2(NMHC)","NOx(GT)","PT08.S3(NOx)","NO2(GT)","PT08.S4(NO2)", "PT08.S5(O3)", "T","RH",]].astype("float64")

# 정답값 데이터프레임 분리
data_y = data_train["AH"].astype("float64")
data_y_val = validation_data["AH"].astype("float64")

# 데이터 형태 출력
print(data_x.shape)
print(data_y.shape)


In [ ]:
# 모델 생성
clf = ak.TimeseriesForecaster(
    lookback=3, predict_from=1, predict_until=10,
    max_trials=1, objective="val_loss",
)

# 모델 훈련
clf.fit(
    x=data_x, y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32, epochs=10,
)


In [ ]:
# 모델 예측
predictions = clf.predict(data_x_test)

# 모델 평가
print(clf.evaluate(data_x_val, data_y_val))